In [43]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 24: Immune System Simulator 20XX ---</h2><p>After <a href="https://www.youtube.com/watch?v=NDVjLt_QHL8&amp;t=7" target="_blank">a weird buzzing noise</a>, you appear back at the man's cottage. He seems relieved to see his friend, but quickly notices that the little reindeer caught some kind of cold while out exploring.</p>
<p>The portly man explains that this reindeer's immune system isn't similar to regular reindeer immune systems:</p>
<p>The <em>immune system</em> and the <em>infection</em> each have <span title="On second thought, it's pretty similar to regular reindeer immune systems.">an army</span> made up of several <em>groups</em>; each <em>group</em> consists of one or more identical <em>units</em>.  The armies repeatedly <em>fight</em> until only one army has units remaining.</p>
<p><em>Units</em> within a group all have the same <em>hit points</em> (amount of damage a unit can take before it is destroyed), <em>attack damage</em> (the amount of damage each unit deals), an <em>attack type</em>, an <em>initiative</em> (higher initiative units attack first and win ties), and sometimes <em>weaknesses</em> or <em>immunities</em>. Here is an example group:</p>
<pre><code>18 units each with 729 hit points (weak to fire; immune to cold, slashing)
 with an attack that does 8 radiation damage at initiative 10
</code></pre>
<p>Each group also has an <em>effective power</em>: the number of units in that group multiplied by their attack damage. The above group has an effective power of <code>18 * 8 = 144</code>. Groups never have zero or negative units; instead, the group is removed from combat.</p>
<p>Each <em>fight</em> consists of two phases: <em>target selection</em> and <em>attacking</em>.</p>
<p>During the <em>target selection</em> phase, each group attempts to choose one target. In decreasing order of effective power, groups choose their targets; in a tie, the group with the higher initiative chooses first. The attacking group chooses to target the group in the enemy army to which it would deal the most damage (after accounting for weaknesses and immunities, but not accounting for whether the defending group has enough units to actually receive all of that damage).</p>
<p>If an attacking group is considering two defending groups to which it would deal equal damage, it chooses to target the defending group with the largest effective power; if there is still a tie, it chooses the defending group with the highest initiative.  If it cannot deal any defending groups damage, it does not choose a target.  Defending groups can only be chosen as a target by one attacking group.</p>
<p>At the end of the target selection phase, each group has selected zero or one groups to attack, and each group is being attacked by zero or one groups.</p>
<p>During the <em>attacking</em> phase, each group deals damage to the target it selected, if any. Groups attack in decreasing order of initiative, regardless of whether they are part of the infection or the immune system. (If a group contains no units, it cannot attack.)</p>
<p>The damage an attacking group deals to a defending group depends on the attacking group's attack type and the defending group's immunities and weaknesses.  By default, an attacking group would deal damage equal to its <em>effective power</em> to the defending group.  However, if the defending group is <em>immune</em> to the attacking group's attack type, the defending group instead takes <em>no damage</em>; if the defending group is <em>weak</em> to the attacking group's attack type, the defending group instead takes <em>double damage</em>.</p>
<p>The defending group only loses <em>whole units</em> from damage; damage is always dealt in such a way that it kills the most units possible, and any remaining damage to a unit that does not immediately kill it is ignored. For example, if a defending group contains <code>10</code> units with <code>10</code> hit points each and receives <code>75</code> damage, it loses exactly <code>7</code> units and is left with <code>3</code> units at full health.</p>
<p>After the fight is over, if both armies still contain units, a new fight begins; combat only ends once one army has lost all of its units.</p>
<p>For example, consider the following armies:</p>
<pre><code>Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with
 an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning,
 slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack
that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire,
cold) with an attack that does 12 slashing damage at initiative 4
</code></pre>

<p>If these armies were to enter combat, the following fights, including details during the target selection and attacking phases, would take place:</p>
<pre><code>Immune System:
Group 1 contains 17 units
Group 2 contains 989 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

Infection group 1 would deal defending group 1 185832 damage
Infection group 1 would deal defending group 2 185832 damage
Infection group 2 would deal defending group 2 107640 damage
Immune System group 1 would deal defending group 1 76619 damage
Immune System group 1 would deal defending group 2 153238 damage
Immune System group 2 would deal defending group 1 24725 damage

Infection group 2 attacks defending group 2, killing 84 units
Immune System group 2 attacks defending group 1, killing 4 units
Immune System group 1 attacks defending group 2, killing 51 units
Infection group 1 attacks defending group 1, killing 17 units
</code></pre>

<pre><code>Immune System:
Group 2 contains 905 units
Infection:
Group 1 contains 797 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 184904 damage
Immune System group 2 would deal defending group 1 22625 damage
Immune System group 2 would deal defending group 2 22625 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 144 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 761 units
Infection:
Group 1 contains 793 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183976 damage
Immune System group 2 would deal defending group 1 19025 damage
Immune System group 2 would deal defending group 2 19025 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 143 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 618 units
Infection:
Group 1 contains 789 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183048 damage
Immune System group 2 would deal defending group 1 15450 damage
Immune System group 2 would deal defending group 2 15450 damage

Immune System group 2 attacks defending group 1, killing 3 units
Infection group 1 attacks defending group 2, killing 143 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 475 units
Infection:
Group 1 contains 786 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 182352 damage
Immune System group 2 would deal defending group 1 11875 damage
Immune System group 2 would deal defending group 2 11875 damage

Immune System group 2 attacks defending group 1, killing 2 units
Infection group 1 attacks defending group 2, killing 142 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 333 units
Infection:
Group 1 contains 784 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181888 damage
Immune System group 2 would deal defending group 1 8325 damage
Immune System group 2 would deal defending group 2 8325 damage

Immune System group 2 attacks defending group 1, killing 1 unit
Infection group 1 attacks defending group 2, killing 142 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 191 units
Infection:
Group 1 contains 783 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181656 damage
Immune System group 2 would deal defending group 1 4775 damage
Immune System group 2 would deal defending group 2 4775 damage

Immune System group 2 attacks defending group 1, killing 1 unit
Infection group 1 attacks defending group 2, killing 142 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 49 units
Infection:
Group 1 contains 782 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181424 damage
Immune System group 2 would deal defending group 1 1225 damage
Immune System group 2 would deal defending group 2 1225 damage

Immune System group 2 attacks defending group 1, killing 0 units
Infection group 1 attacks defending group 2, killing 49 units
</code></pre>
<pre><code>Immune System:
No groups remain.
Infection:
Group 1 contains 782 units
Group 2 contains 4434 units
</code></pre>
<p>In the example above, the winning army ends up with <code>782 + 4434 = <em>5216</em></code> units.</p>
<p>You scan the reindeer's condition (your puzzle input); the white-bearded man looks nervous.  As it stands now, <em>how many units would the winning army have</em>?</p>
</article>


In [44]:
from copy import deepcopy
from enum import Enum, auto
from re import compile, findall, split

from more_itertools import first, one

example = """
Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack  that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
"""
DO_PRINT = False


class Weapon(Enum):
    bludgeoning = auto()
    cold = auto()
    fire = auto()
    radiation = auto()
    slashing = auto()

    def __repr__(self):
        return f"{self.name}"


@dataclass
class Group:
    nr: int
    units: int
    hit_points: int  # (amount of damage a unit can take before it is destroyed)
    attack_damage: int  # (the amount of damage each unit deals)
    attack_type: Weapon
    initiative: int
    immunities: list[Weapon]
    weaknesses: list[Weapon]

    @property
    def effective_power(self) -> int:
        return self.units * self.attack_damage

    def damage(self, defender: Group) -> int:
        # The damage an attacking group deals to a defending group depends on
        # the attacking group's attack type and
        # the defending group's immunities and weaknesses.
        if self.attack_type in defender.immunities:
            # However, if the defending group is immune to the attacking group's
            # attack type, the defending group instead takes no damage;
            return 0

        if self.attack_type in defender.weaknesses:
            # if the defending group is weak to the attacking group's attack type,
            # the defending group instead takes double damage.
            return 2 * self.effective_power

        # By default, an attacking group would deal damage equal to its
        # effective power to the defending group.
        return self.effective_power

    def select_target(self, defenders: list[Group]) -> Group | None:
        # If an attacking group is considering two defending groups to which it
        # would deal equal damage,
        # it chooses to target the defending group
        # with the largest effective power;
        # if there is still a tie,
        # it chooses the defending group with the highest initiative.
        # If it cannot deal any defending groups damage,
        # it does not choose a target.
        # Defending groups can only be chosen as a target by one attacking group.
        damage, candidate = first(
            sorted(
                ((self.damage(defender), defender) for defender in defenders),
                key=lambda t: (-t[0], -t[1].effective_power, -t[1].initiative),
            ),
            (0, None),
        )
        if damage == 0:
            return None

        defenders.remove(candidate)
        return candidate

    def attack_target_str(self, units_killed: int, candidate: Group) -> str:
        raise ("get_str should not be called on Group!")

    def select_target_str(self, damage: int, candidate: Group) -> str:
        raise ("get_str should not be called on Group!")

    @classmethod
    def parse(cls, nr: int, s: str) -> Group:
        attack_regex = compile(rf'\d+\s({r"|".join(Weapon._member_names_)})')

        units, hit_points, attack_damage, initiative = (
            int(i) for i in findall(r"-?\d+", s)
        )
        attack_type = Weapon[one(findall(attack_regex, s))]
        weaknesses, immunities = [], []
        for o in findall(r"\(.*?\)", s):
            for oo in o[1:-1].split("; "):
                if oo.startswith("weak"):
                    weaknesses = {
                        Weapon[w] for w in oo.removeprefix("weak to ").split(", ")
                    }
                else:
                    immunities = {
                        Weapon[w] for w in oo.removeprefix("immune to ").split(", ")
                    }

        return cls(
            nr=nr,
            units=units,
            hit_points=hit_points,
            attack_damage=attack_damage,
            attack_type=attack_type,
            initiative=initiative,
            immunities=immunities,
            weaknesses=weaknesses,
        )


class Immunity(Group):
    def attack_target_str(self, units_lost: int, defender: Group) -> str:
        return f"Immune System group {self.nr} would deal defending group {defender.nr}, killing {units_lost} units"

    def __repr__(self) -> str:
        s = super().__repr__()
        i = s.find(",")
        return s[:i] + f", effective_power={self.effective_power}" + s[i:]


class Infection(Group):
    def attack_target_str(self, units_lost: int, defender: Group) -> str:
        return f"Infection group {self.nr} attacks defending group {defender.nr}, killing {units_lost} units"

    def __repr__(self) -> str:
        s = super().__repr__()
        i = s.find(",")
        return s[:i] + f", effective_power={self.effective_power}" + s[i:]


class System:
    def __init__(self, immune: list[Group], infection: list[Group]) -> None:
        self.immune = immune
        self.infection = infection
        self.do_print = False

    def target_selection(self) -> list[tuple[Group, Group]]:
        # During the target selection phase,
        # - Each group attempts to choose one target.
        # - In decreasing order of effective power, decreasing order of initiative chooses.
        #
        # The attacking group chooses to target the group in the enemy army to
        # which it would deal the most damage (after accounting for weaknesses
        # and immunities, but not accounting for whether the defending group has
        # enough units to actually receive all of that damage).
        groups = sorted(
            self.immune + self.infection,
            key=lambda g: (-g.effective_power, -g.initiative),
        )

        immune = self.immune[:]
        infection = self.infection[:]

        return [
            (g, d)
            for g in groups
            if (d := g.select_target(immune if isinstance(g, Infection) else infection))
            is not None
        ]

    def attack(self, attacks: list[tuple[Group, Group]]) -> None:
        attacks.sort(key=lambda t: -t[0].initiative)
        change = False
        for attacker, defender in attacks:
            damage = attacker.damage(defender)
            units_lost = min(defender.units, damage // defender.hit_points)
            defender.units -= units_lost
            change |= units_lost > 0
            if DO_PRINT:
                print(attacker.attack_target_str(units_lost, defender))
        return change

    def fight(self):
        change = True
        while change and self.immune and self.infection:
            if DO_PRINT:
                print(self)
                print()

            attacks = self.target_selection()
            change = self.attack(attacks)

            if DO_PRINT:
                print()
                print()

            self.immune = [i for i in self.immune if i.units > 0]
            self.infection = [i for i in self.infection if i.units > 0]

        if DO_PRINT:
            print(self)
            print()

        return sum(g.units for g in self.immune + self.infection)

    def give_boost(self, boost: int) -> System:
        boosted_system = deepcopy(self)
        for im in boosted_system.immune:
            im.attack_damage += boost
        return boosted_system

    def reindeer_lives(self) -> bool:
        return bool(self.immune) and not self.infection

    def __repr__(self) -> str:
        return "\n".join(
            [
                "Immune System:",
                "\n".join(
                    (f"Group {im.nr} contains {im.units} units" for im in self.immune)
                    if self.immune
                    else ("No groups remain." for _ in range(1))
                ),
                "Infection:",
                "\n".join(
                    (
                        f"Group {im.nr} contains {im.units} units"
                        for im in self.infection
                    )
                    if self.infection
                    else ("No groups remain." for _ in range(1))
                ),
                "",
                "\n".join(
                    (
                        f"Infection group {a.nr} would deal defending group {d.nr} {a.damage(d)} damage"
                        for a, d in product(s.infection, s.immune)
                    ),
                ),
                "\n".join(
                    (
                        f"Immune group {a.nr} would deal defending group {d.nr} {a.damage(d)} damage"
                        for a, d in product(s.immune, s.infection)
                    ),
                ),
            ]
        )

    @classmethod
    def units_left_smallest_boost(cls, s: str) -> int:
        base = cls.parse(s)

        high = 1
        reindeer_won = False
        factor = 10
        while not reindeer_won:
            bs = base.give_boost(high)
            bs.fight()
            high *= factor
            reindeer_won = bs.reindeer_lives()

        high //= factor
        low = high // factor

        mid = 0
        while low <= high:
            mid = (low + high) // 2
            bs = base.give_boost(mid)
            bs.fight()
            if bs.reindeer_lives():
                best = bs
                high = mid - 1
            else:
                low = mid + 1
        return sum(b.units for b in best.immune)

    @classmethod
    def parse(cls, s: str) -> System:
        immune, infection = split(r"\n\s*\n", s.strip())
        immune_groups = [
            Immunity.parse(i, ig)
            for i, ig in enumerate(immune.splitlines()[1:], start=1)
        ]
        infection_groups = [
            Infection.parse(i, ig)
            for i, ig in enumerate(infection.splitlines()[1:], start=1)
        ]
        system = System(immune_groups, infection_groups)
        return system


s = System.parse(example)
print(f"Example: {s.fight()} should be 5216")

Example: 5216 should be 5216


In [45]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

s = System.parse(puzzle)
print(f"Part I: {s.fight()}")

Part I: 16678


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>16678</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Things aren't looking good for the reindeer. The man asks whether more milk and cookies would help you think.</p>
<p>If only you could give the reindeer's immune system a boost, you might be able to change the outcome of the combat.</p>
<p>A <em>boost</em> is an integer increase in immune system units' attack damage. For example, if you were to boost the above example's immune system's units by <code>1570</code>, the armies would instead look like this:</p>
<pre><code>Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with
 an attack that does <em>6077</em> fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning,
 slashing) with an attack that does <em>1595</em> slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack
that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire,
cold) with an attack that does 12 slashing damage at initiative 4
</code></pre>

<p>With this boost, the combat proceeds differently:</p>
<pre><code>Immune System:
Group 2 contains 989 units
Group 1 contains 17 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

Infection group 1 would deal defending group 2 185832 damage
Infection group 1 would deal defending group 1 185832 damage
Infection group 2 would deal defending group 1 53820 damage
Immune System group 2 would deal defending group 1 1577455 damage
Immune System group 2 would deal defending group 2 1577455 damage
Immune System group 1 would deal defending group 2 206618 damage

Infection group 2 attacks defending group 1, killing 9 units
Immune System group 2 attacks defending group 1, killing 335 units
Immune System group 1 attacks defending group 2, killing 32 units
Infection group 1 attacks defending group 2, killing 84 units
</code></pre>

<pre><code>Immune System:
Group 2 contains 905 units
Group 1 contains 8 units
Infection:
Group 1 contains 466 units
Group 2 contains 4453 units

Infection group 1 would deal defending group 2 108112 damage
Infection group 1 would deal defending group 1 108112 damage
Infection group 2 would deal defending group 1 53436 damage
Immune System group 2 would deal defending group 1 1443475 damage
Immune System group 2 would deal defending group 2 1443475 damage
Immune System group 1 would deal defending group 2 97232 damage

Infection group 2 attacks defending group 1, killing 8 units
Immune System group 2 attacks defending group 1, killing 306 units
Infection group 1 attacks defending group 2, killing 29 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 876 units
Infection:
Group 2 contains 4453 units
Group 1 contains 160 units

Infection group 2 would deal defending group 2 106872 damage
Immune System group 2 would deal defending group 2 1397220 damage
Immune System group 2 would deal defending group 1 1397220 damage

Infection group 2 attacks defending group 2, killing 83 units
Immune System group 2 attacks defending group 2, killing 427 units
</code></pre>
<p>After a few fights...</p>
<pre><code>Immune System:
Group 2 contains 64 units
Infection:
Group 2 contains 214 units
Group 1 contains 19 units

Infection group 2 would deal defending group 2 5136 damage
Immune System group 2 would deal defending group 2 102080 damage
Immune System group 2 would deal defending group 1 102080 damage

Infection group 2 attacks defending group 2, killing 4 units
Immune System group 2 attacks defending group 2, killing 32 units
</code></pre>

<pre><code>Immune System:
Group 2 contains 60 units
Infection:
Group 1 contains 19 units
Group 2 contains 182 units

Infection group 1 would deal defending group 2 4408 damage
Immune System group 2 would deal defending group 1 95700 damage
Immune System group 2 would deal defending group 2 95700 damage

Immune System group 2 attacks defending group 1, killing 19 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 60 units
Infection:
Group 2 contains 182 units

Infection group 2 would deal defending group 2 4368 damage
Immune System group 2 would deal defending group 2 95700 damage

Infection group 2 attacks defending group 2, killing 3 units
Immune System group 2 attacks defending group 2, killing 30 units
</code></pre>
<p>After a few more fights...</p>
<pre><code>Immune System:
Group 2 contains 51 units
Infection:
Group 2 contains 40 units

Infection group 2 would deal defending group 2 960 damage
Immune System group 2 would deal defending group 2 81345 damage

Infection group 2 attacks defending group 2, killing 0 units
Immune System group 2 attacks defending group 2, killing 27 units
</code></pre>

<pre><code>Immune System:
Group 2 contains 51 units
Infection:
Group 2 contains 13 units

Infection group 2 would deal defending group 2 312 damage
Immune System group 2 would deal defending group 2 81345 damage

Infection group 2 attacks defending group 2, killing 0 units
Immune System group 2 attacks defending group 2, killing 13 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 51 units
Infection:
No groups remain.
</code></pre>
<p>This boost would allow the immune system's armies to win! It would be left with <code><em>51</em></code> units.</p>
<p>You don't even know <em>how</em> you could boost the reindeer's immune system or what effect it might have, so you need to be cautious and find the <em>smallest boost</em> that would allow the immune system to win.</p>
<p><em>How many units does the immune system have left</em> after getting the smallest boost it needs to win?</p>
</article>

</main>


In [46]:
DO_PRINT = False
print(f"Example: {System.units_left_smallest_boost(example)} should be 51")

Example: 51 should be 51


In [47]:
print(f"Part II: {System.units_left_smallest_boost(puzzle)}")

Part II: 3758


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3758</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
